In [ ]:
import re


In [ ]:
class Map:
    """Mapping from one type to another"""
    def __init__(self, mappings: dict, output: str):
        self.mappings = mappings
        self.output = output

    def convert(self, value):
        for source_range, diff in self.mappings.items():
            if value in source_range:
                return value + diff, self.output
        return value, self.output
    
    @classmethod
    def from_rules(cls, rules: list[str], output: str):
        mappings = dict()
        for rule in rules:
            d, s, r = map(int, rule.split(" "))
            mappings[range(s, s + r)] = d - s
        return cls(mappings, output)


In [ ]:
from collections import defaultdict

data = defaultdict(list)
with open("day05_example.txt") as file:
    for line in file:
        if match := re.match("seeds: (.+)", line):
            seeds = [int(s) for s in match.group(1).split(" ")]
        elif match := re.match("(\w+)-to-(\w+)", line):
            source, dest = match.groups()
        elif match := re.match("(\d+) (\d+) (\d+)", line):
            data[(source, dest)].append(line.strip())


In [ ]:
maps = dict()
for (source, dest), rules in data.items():
    maps[source] = Map.from_rules(rules, dest)


# Part 1


In [ ]:
locations = list()

for value in seeds:
    output = "seed"
    while True:
        value, output = maps[output].convert(value)
        # print(output, value)
        if not output in maps:
            locations.append(value)
            break

print("Answer:", min(locations))


# Part 2


In [ ]:
locations = []

for start, length in zip(seeds[::2], seeds[1::2]):
    print("Range:", start, length)
    for value in range(start, start + length):
        output = "seed"
        while True:
            value, output = maps[output].convert(value)
            # print(output, value)
            if not output in maps:
                locations.append(value)
                break

print("Answer:", min(locations))

